In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torch.optim import lr_scheduler

import torchvision
import torchvision.models as models
import torchvision.datasets as dset
import torchvision.transforms as transforms

import random as rand
from random import *
import os
import cv2
import copy
import time
from PIL import Image

In [4]:
train_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
}

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
data_dir = "../../Data_Set/Labeld_Crop_Data/"
trDsets = {x: dset.ImageFolder(os.path.join(data_dir, x), train_transforms[x]) for x in ['train', 'val']}
trLoaders = {x: torch.utils.data.DataLoader(trDsets[x], batch_size=64, shuffle=True, num_workers=4) for x in ['train', 'val']}

teDsets = dset.ImageFolder(os.path.join(data_dir, 'test'), transform=test_transforms)
teLoaders = torch.utils.data.DataLoader(teDsets, batch_size=64, shuffle=False, num_workers=4)

In [6]:
trDsets_sizes = {x: len(trDsets[x]) for x in ['train', 'val']}
class_names = trDsets['train'].classes

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

In [8]:
model = models.resnext50_32x4d(pretrained=False, num_classes=22)
model = model.to(device)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, factor=0.1, patience=11)

In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=8, device=device):
    
    global_info = []
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    early_stopping = EarlyStopping(patience=11, verbose=True)
    for epoch in range(num_epochs):
        local_info = []
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
                if epoch > 0:
                    scheduler.step(val_loss)
                    
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in trLoaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / trDsets_sizes[phase]
            if phase == 'val':
                val_loss = running_loss / trDsets_sizes['val']
            epoch_acc = running_corrects.double() / trDsets_sizes[phase]

            if phase == 'train':
                local_info.append(epoch_loss)
                ea = epoch_acc.cpu().numpy()
                local_info.append(ea)
            else:
                local_info.append(epoch_loss)
                ea = epoch_acc.cpu().numpy()
                local_info.append(ea)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        lr_get = get_lr(optimizer)
        print("Current learning rate : {:.8f}".format(lr_get))
        global_info.append(local_info)
        
        if phase =='val':
            early_stopping(epoch_loss, model)

            if early_stopping.early_stop:
                print("Early stopping")
                break

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
        
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [11]:
torch.cuda.empty_cache()
model_ft = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=100, device=device)

Epoch 0/99
----------
train Loss: 2.5219 Acc: 0.2323
val Loss: 1.7672 Acc: 0.4471
Current learning rate : 0.00100000
Validation loss decreased (inf --> 1.767185).  Saving model ...
Epoch 1/99
----------
train Loss: 1.3464 Acc: 0.5762
val Loss: 0.9120 Acc: 0.7048
Current learning rate : 0.00100000
Validation loss decreased (1.767185 --> 0.912002).  Saving model ...
Epoch 2/99
----------
train Loss: 0.7442 Acc: 0.7634
val Loss: 0.5087 Acc: 0.8466
Current learning rate : 0.00100000
Validation loss decreased (0.912002 --> 0.508685).  Saving model ...
Epoch 3/99
----------
train Loss: 0.4799 Acc: 0.8489
val Loss: 0.3202 Acc: 0.9073
Current learning rate : 0.00100000
Validation loss decreased (0.508685 --> 0.320238).  Saving model ...
Epoch 4/99
----------
train Loss: 0.3219 Acc: 0.9019
val Loss: 0.2485 Acc: 0.9281
Current learning rate : 0.00100000
Validation loss decreased (0.320238 --> 0.248489).  Saving model ...
Epoch 5/99
----------
train Loss: 0.2604 Acc: 0.9232
val Loss: 0.1833 Acc: 

In [12]:
torch.save(model_ft, 'lotte_model_resnext.pt')

In [13]:
testModel = torch.load('lotte_model_resnext.pt', map_location=device)

In [14]:
correct = 0
total = 0
with torch.no_grad():
    for data in teLoaders:
        images, labels = data
        images, labels = Variable(images.float().cuda()), Variable(labels.float().cuda())
        
        outputs = testModel(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of test images: %.2f %%' % (100 * correct / total))

Accuracy of test images: 98.70 %


In [15]:
tag_classes = ['ID_gum', 'buttering', 'couque_coffee', 'chocopie', 'cidar', 'couque_white', 'coke', 'diget_ori', 'diget_choco', 'gumi_gumi', 'homerunball', 'jjolbyung_noodle', 'juicyfresh', 'jjolbyung_ori', 'spearmint', 'squid_peanut', 'samdasu', 'tuna', 'toreta', 'vita500', 'welchs', 'zec']
tag_dict = dict()
for i, label in enumerate(tag_classes):
    tag_dict[i] = label

print(tag_dict)

{0: 'ID_gum', 1: 'buttering', 2: 'couque_coffee', 3: 'chocopie', 4: 'cidar', 5: 'couque_white', 6: 'coke', 7: 'diget_ori', 8: 'diget_choco', 9: 'gumi_gumi', 10: 'homerunball', 11: 'jjolbyung_noodle', 12: 'juicyfresh', 13: 'jjolbyung_ori', 14: 'spearmint', 15: 'squid_peanut', 16: 'samdasu', 17: 'tuna', 18: 'toreta', 19: 'vita500', 20: 'welchs', 21: 'zec'}


In [17]:
class_correct = list(0 for i in range(len(class_names)))
class_total = list(0 for i in range(len(class_names)))
with torch.no_grad():
    for data in teLoaders:
        images, labels = data
        images, labels = Variable(images.float().cuda()), Variable(labels.float().cuda())
        
        outputs = testModel(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()

        for i in range(c.size(0)):
            label = labels[i]
            class_correct[int(label.item())] += c[i].item()
            class_total[int(label.item())] += 1

for i in range(len(class_names)):
    print('Accuracy of %5s : %.2f %%' % (tag_dict[int(class_names[i])], 100 * class_correct[i] / class_total[i]))

Accuracy of ID_gum : 98.28 %
Accuracy of buttering : 100.00 %
Accuracy of homerunball : 100.00 %
Accuracy of jjolbyung_noodle : 100.00 %
Accuracy of juicyfresh : 100.00 %
Accuracy of jjolbyung_ori : 100.00 %
Accuracy of spearmint : 97.70 %
Accuracy of squid_peanut : 100.00 %
Accuracy of samdasu : 99.51 %
Accuracy of  tuna : 98.99 %
Accuracy of toreta : 98.72 %
Accuracy of vita500 : 99.47 %
Accuracy of couque_coffee : 99.57 %
Accuracy of welchs : 100.00 %
Accuracy of   zec : 98.65 %
Accuracy of chocopie : 98.60 %
Accuracy of cidar : 99.60 %
Accuracy of couque_white : 97.98 %
Accuracy of  coke : 98.45 %
Accuracy of diget_ori : 96.99 %
Accuracy of diget_choco : 92.15 %
Accuracy of gumi_gumi : 100.00 %


## Normalized and Augmentation

In [3]:
train_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomChoice([
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=(0.5, 1.5)),
            transforms.ColorJitter(contrast=(0.5, 1.5)),
        ]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [4]:
data_dir = "../../Data_Set/Labeld_Crop_Data/"
trDsets = {x: dset.ImageFolder(os.path.join(data_dir, x), train_transforms[x]) for x in ['train', 'val']}
trLoaders = {x: torch.utils.data.DataLoader(trDsets[x], batch_size=64, shuffle=True, num_workers=4) for x in ['train', 'val']}

teDsets = dset.ImageFolder(os.path.join(data_dir, 'test'), transform=test_transforms)
teLoaders = torch.utils.data.DataLoader(teDsets, batch_size=64, shuffle=False, num_workers=4)

trDsets_sizes = {x: len(trDsets[x]) for x in ['train', 'val']}
class_names = trDsets['train'].classes

In [7]:
model = models.resnext50_32x4d(pretrained=False, num_classes=22)
model = model.to(device)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, factor=0.1, patience=11)

In [9]:
torch.cuda.empty_cache()
model_ft = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=100, device=device)

Epoch 0/99
----------
train Loss: 2.7083 Acc: 0.1748
val Loss: 1.9544 Acc: 0.3983
Current learning rate : 0.00100000
Validation loss decreased (inf --> 1.954406).  Saving model ...
Epoch 1/99
----------
train Loss: 1.6092 Acc: 0.4673
val Loss: 0.9661 Acc: 0.6776
Current learning rate : 0.00100000
Validation loss decreased (1.954406 --> 0.966142).  Saving model ...
Epoch 2/99
----------
train Loss: 0.9558 Acc: 0.6854
val Loss: 0.5845 Acc: 0.8182
Current learning rate : 0.00100000
Validation loss decreased (0.966142 --> 0.584451).  Saving model ...
Epoch 3/99
----------
train Loss: 0.6528 Acc: 0.7969
val Loss: 0.3995 Acc: 0.8622
Current learning rate : 0.00100000
Validation loss decreased (0.584451 --> 0.399544).  Saving model ...
Epoch 4/99
----------
train Loss: 0.4981 Acc: 0.8475
val Loss: 0.2955 Acc: 0.9033
Current learning rate : 0.00100000
Validation loss decreased (0.399544 --> 0.295455).  Saving model ...
Epoch 5/99
----------
train Loss: 0.4082 Acc: 0.8715
val Loss: 0.2631 Acc: 

train Loss: 0.0411 Acc: 0.9861
val Loss: 0.0655 Acc: 0.9888
Current learning rate : 0.00100000
EarlyStopping counter: 3 out of 11
Epoch 50/99
----------
train Loss: 0.0537 Acc: 0.9818
val Loss: 0.0806 Acc: 0.9852
Current learning rate : 0.00100000
EarlyStopping counter: 4 out of 11
Epoch 51/99
----------
train Loss: 0.0375 Acc: 0.9887
val Loss: 0.0651 Acc: 0.9892
Current learning rate : 0.00100000
EarlyStopping counter: 5 out of 11
Epoch 52/99
----------
train Loss: 0.0515 Acc: 0.9831
val Loss: 0.0684 Acc: 0.9876
Current learning rate : 0.00100000
EarlyStopping counter: 6 out of 11
Epoch 53/99
----------
train Loss: 0.0476 Acc: 0.9840
val Loss: 0.0690 Acc: 0.9888
Current learning rate : 0.00100000
EarlyStopping counter: 7 out of 11
Epoch 54/99
----------
train Loss: 0.0400 Acc: 0.9883
val Loss: 0.0645 Acc: 0.9896
Current learning rate : 0.00100000
EarlyStopping counter: 8 out of 11
Epoch 55/99
----------
train Loss: 0.0301 Acc: 0.9910
val Loss: 0.0684 Acc: 0.9912
Current learning rate 

In [10]:
torch.save(model_ft, 'lotte_model_resnext2.pt')

In [11]:
testModel = torch.load('lotte_model_resnext2.pt', map_location=device)

In [12]:
correct = 0
total = 0
with torch.no_grad():
    for data in teLoaders:
        images, labels = data
        images, labels = Variable(images.float().cuda()), Variable(labels.float().cuda())
        
        outputs = testModel(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of test images: %.2f %%' % (100 * correct / total))

Accuracy of test images: 99.01 %


In [13]:
tag_classes = ['ID_gum', 'buttering', 'couque_coffee', 'chocopie', 'cidar', 'couque_white', 'coke', 'diget_ori', 'diget_choco', 'gumi_gumi', 'homerunball', 'jjolbyung_noodle', 'juicyfresh', 'jjolbyung_ori', 'spearmint', 'squid_peanut', 'samdasu', 'tuna', 'toreta', 'vita500', 'welchs', 'zec']
tag_dict = dict()
for i, label in enumerate(tag_classes):
    tag_dict[i] = label

print(tag_dict)

{0: 'ID_gum', 1: 'buttering', 2: 'couque_coffee', 3: 'chocopie', 4: 'cidar', 5: 'couque_white', 6: 'coke', 7: 'diget_ori', 8: 'diget_choco', 9: 'gumi_gumi', 10: 'homerunball', 11: 'jjolbyung_noodle', 12: 'juicyfresh', 13: 'jjolbyung_ori', 14: 'spearmint', 15: 'squid_peanut', 16: 'samdasu', 17: 'tuna', 18: 'toreta', 19: 'vita500', 20: 'welchs', 21: 'zec'}


In [15]:
class_correct = list(0 for i in range(len(class_names)))
class_total = list(0 for i in range(len(class_names)))
with torch.no_grad():
    for data in teLoaders:
        images, labels = data
        images, labels = Variable(images.float().cuda()), Variable(labels.float().cuda())
        
        outputs = testModel(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()

        for i in range(c.size(0)):
            label = labels[i]
            class_correct[int(label.item())] += c[i].item()
            class_total[int(label.item())] += 1

for i in range(len(class_names)):
    print('Accuracy of %5s : %.5f %%' % (tag_dict[int(class_names[i])], 100 * class_correct[i] / class_total[i]))

Accuracy of ID_gum : 98.71245 %
Accuracy of buttering : 100.00000 %
Accuracy of homerunball : 100.00000 %
Accuracy of jjolbyung_noodle : 100.00000 %
Accuracy of juicyfresh : 100.00000 %
Accuracy of jjolbyung_ori : 100.00000 %
Accuracy of spearmint : 98.27586 %
Accuracy of squid_peanut : 100.00000 %
Accuracy of samdasu : 100.00000 %
Accuracy of  tuna : 99.66443 %
Accuracy of toreta : 100.00000 %
Accuracy of vita500 : 100.00000 %
Accuracy of couque_coffee : 99.56897 %
Accuracy of welchs : 100.00000 %
Accuracy of   zec : 98.98649 %
Accuracy of chocopie : 98.59649 %
Accuracy of cidar : 99.60317 %
Accuracy of couque_white : 99.19355 %
Accuracy of  coke : 99.22481 %
Accuracy of diget_ori : 93.98496 %
Accuracy of diget_choco : 94.86405 %
Accuracy of gumi_gumi : 100.00000 %
